# Python with SQLite

* SQLite3 is a very easy to use database engine. 
 * It is self-contained, serverless, zero-configuration and transactional. 
 * It is very fast and lightweight, and the entire database is stored in a single disk file. 
 * It is used in a lot of applications as internal data storage. 
* The Python Standard Library includes a module called "sqlite3" intended for working with this database. This module is a SQL interface compliant with the DB-API 2.0 specification.

## Using Python's SQLlite Module

In [1]:
import sqlite3

## Connecting SQLite to the Database
The following creates or opens a file called mydb with a SQLite3 DB. 

In [2]:
db = sqlite3.connect('data/mydb.db', 
                     detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)

When done working with the DB we need to close the connection:

In [3]:
db.close()

## Creating (CREATE) and Deleting (DROP) Tables
* To operate on the database we require a cursor object and pass the SQL statments to the cursor object to execute them.
* Finally, it is neessary to commit the changes.
* We will create a `users` table with `name, phone, email and password` columns

In [3]:
cursor = db.cursor()
cursor.execute('''
               CREATE TABLE users(
               id INTEGER PRIMARY KEY, 
               name TEXT,
               phone TEXT,
               email TEXT unique,
               password TEXT)
               ''')
db.commit()

To drop a table:

In [16]:
cursor = db.cursor()
cursor.execute('''DROP TABLE users''')
db.commit()

Please note that the commit function is invoked on the db object, not the cursor object.

## Inserting (INSERT) Data into the Database
* To insert data we use the cursor to execute the query. If you need values from Python variables it is recommended to use the "?" placeholder. 
* Never use string operations or concatenation to make your queries because is very insecure. 
* In this example we are going to insert two users in the database, their information is stored in python variables.

In [4]:
cursor = db.cursor()
name1 = "Atul"
phone1 = "9338703277"
email1 = "anag@suiit.ac.in"
password1 = "12345"

name2 = "Nag"
phone2 = "7078711408"
email2 = "nag.atul@gmail.com"
password2 = "78910"

name3 = "PythonNewbie"
phone3 = "123456"
email3 = "my@mydomain.com"
password3 = "123456"

In [5]:
cursor.execute('''INSERT INTO users(name, phone, email, password)
                  VALUES(?,?,?,?)''',(name1, phone1, email1, password1))
print('First user inserted')
db.commit()

First user inserted


The values of the Python variables are passed inside a tuple. Another way to do this is passing a dictionary using the `":keyname"` placeholder:

In [6]:
cursor.execute('''INSERT INTO users(name, phone, email, password)
                  VALUES(:name,:phone, :email, :password)''',
                  {'name':name2, 'phone':phone2,'email':email2,
                   'password':password2})
db.commit()

If you need to insert several users use `executemany` and a list with the tuples:

In [68]:
users = [(name1, phone1, email1, password1),
        (name2, phone2, email2, password2),
        (name3, phone3, email3, password3)]
cursor.executemany('''INSERT INTO users(name, phone, email, password)
                      VALUES(?,?,?,?)''', users)
db.commit()

If you need to get the id of the row you just inserted use `lastrowid` :

In [51]:
id = cursor.lastrowid
print('Last row id:', id)

Last row id: 1


## Retrieving Data (SELECT) with SQLite
To retrieve data, execute the query against the cursor object and then use `fetchone()` to retrieve a single row or `fetchall()` to retrieve all the rows.

In [59]:
cursor.execute('''SELECT name, email, phone FROM users''')
user1 = cursor.fetchone() #retrieve the first row
print(user1[0]) # Print the first column retrieved (user's name)
all_rows = cursor.fetchall()

for row in all_rows:
    print('{0} : {1}, {2}'.format(row[0], row[1], row[2]))

Atul
Nag : nag.atul@gmail.com, 7078711408
PythonNewbie : my@mydomain.com, 123456


The cursor object works as an iterator, invoking `fetchall()` automatically:

In [87]:
cursor.execute('''SELECT name, email, phone FROM users''')
for row in cursor:
    print('{0}: {1}, {2}'.format(row[0], row[1], row[2]))

Atul: anag@suiit.ac.in, 123456
PythonNewbie: my@mydomain.com, 123456


To retrive data with conditions, use again the "?" placeholder:

In [61]:
user_id = '3'
cursor.execute('''SELECT name, email, phone FROM users WHERE id=?''',(user_id))
user = cursor.fetchone()
print(user)

('PythonNewbie', 'my@mydomain.com', '123456')


## Updating and Deleting

In [84]:
newphone = "78910"
userid = 1
cursor.execute('''UPDATE users SET phone = ? WHERE id = ?''',
              (newphone, userid))

#db.commit()

In [77]:
delete_userid = '2'
cursor.execute(''' DELETE FROM users WHERE id = ? ''', (delete_userid))
db.commit()

In [86]:
db.rollback()

##  Database Exceptions

In [7]:
try:
    cursor = db.cursor()
    cursor.execute('''
               CREATE TABLE users(
               id INTEGER PRIMARY KEY, 
               name TEXT,
               phone TEXT,
               email TEXT unique,
               password TEXT)
               ''')
    db.commit()
except Exception as e:
    db.rollback()
    print("The table already exists.")
finally:
    db.close()

The table already exists.


## Date & Time

In [3]:
from datetime import date, datetime

In [4]:
cursor = db.cursor()
cursor.execute('''
                 CREATE TABLE IF NOT EXISTS example(id INTEGER PRIMARY KEY,
                 created_at DATE)
                 ''')
db.commit()

In [10]:
cursor = db.cursor()
cursor.execute('''DROP TABLE example''')
db.commit()

In [5]:
today = date.today()
print(today)
type(today)

2018-11-09


datetime.date

In [8]:
cursor.execute('''INSERT INTO example(created_at) VALUES(?)''',(today,))
db.commit()

In [9]:
cursor.execute('''SELECT created_at FROM example''')
row = cursor.fetchone()
print('The date is {0} and the datatype is {1}'.format(row[0], type(row[0])))

The date is 2018-11-09 and the datatype is <class 'datetime.date'>


In [11]:
cursor = db.cursor()
cursor.execute('''
                 CREATE TABLE IF NOT EXISTS example(id INTEGER PRIMARY KEY,
                 created_at timestamp)
                 ''')
db.commit()

In [12]:
now = datetime.now()
print(now)

2018-11-09 15:28:20.001986


In [13]:
cursor.execute('''INSERT INTO example(created_at) VALUES(?)''',(now,))
db.commit()

In [14]:
cursor.execute('''SELECT created_at FROM example''')
row = cursor.fetchone()
print('The date is {0} and the datatype is {1}'.format(row[0], type(row[0])))

The date is 2018-11-09 15:28:20.001986 and the datatype is <class 'datetime.datetime'>


In [17]:
fh = open('data/sqlqueries.sql','r')
script = fh.read()
cursor.executescript(script)
fh.close()

## Defining SQLite SQL Functions
Sometimes we need to use our own functions in a statement, specially when we
are inserting data in order to accomplish some specific task. A good example of
this is when we are storing passwords in the database and we need to encrypt
those passwords:

In [4]:
import hashlib

In [5]:
def encrypt_password(password):
    encrypted_pass = hashlib.sha1(password.encode('utf-8')).hexdigest()
    return encrypted_pass

#connect

In [6]:
# Register the function
db.create_function('encrypt',1, encrypt_password)
cursor = db.cursor()
cursor.execute('''CREATE TABLE users(
                   id INTEGER PRIMARY KEY,
                   email TEXT,
                   password TEXT)
               ''')
user = ('atul@example.com','12345678')
cursor.execute('''INSERT INTO users(email, password)
                   VALUES (?, encrypt(?))''',user)
db.commit()

In [7]:
cursor.execute('''SELECT id, email, password FROM users''')
for row in cursor:
    print('{0}: {1}, {2}'.format(row[0], row[1], row[2]))

1: atul@example.com, 7c222fb2927d828af22f592134e8932480637c0d
